# Building A Chatbot with History

the chatbot able to have a conversation and remember previous interactions 



In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

from langchain_groq import ChatGroq
model = ChatGroq(
    model="gemma2-9b-it",
    api_key=groq_api_key,
)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x72ef0a9e2680>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x72ef0a824910>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage 
# model.invoke() # -> This message go to ai model 

In [4]:
from langchain_core.messages import AIMessage
model.invoke([HumanMessage(content="Hello my name is omkar i am a Ai engineering student"),
           AIMessage(content="Hello Omkar, nice to meet you!"),
           HumanMessage(content="What is my name?"),]) 

AIMessage(content="Your name is Omkar. 😊  \n\nIt's nice to meet you, Omkar. What can I help you with today?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 45, 'total_tokens': 76, 'completion_time': 0.056363636, 'prompt_time': 0.003615794, 'queue_time': 0.256309855, 'total_time': 0.05997943}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--61bdec8f-8eb3-44fa-9b93-690aff19d1e2-0', usage_metadata={'input_tokens': 45, 'output_tokens': 31, 'total_tokens': 76})

In [ ]:
# Message History 
# we can use message history class to wrap our model and make it stateful , it keep the track of input and output of the model , and store it into datastore , Future interactions will then load those messages and pass them into the chain as part of the input 

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


store  = {}
def get_session_history(session_id : str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(
    model,
    get_session_history
)


In [6]:
config = {"configurable": {"session_id": "chat1"}}


In [7]:
response = with_message_history.invoke(
    [HumanMessage(content="Hello my name is omkar i am a Ai engineering student")],
    config=config
)

In [8]:
response.content

"Hello Omkar!\n\nThat's great to know!  AI engineering is a fascinating field. What are you most interested in learning about or working on within AI?  \n\nI'm happy to chat about anything related to AI, or just have a general conversation.\n"

In [9]:
with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config
)

AIMessage(content='Your name is Omkar.  \n\nYou told me at the beginning of our conversation! 😊  \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 92, 'total_tokens': 115, 'completion_time': 0.041818182, 'prompt_time': 0.00533136, 'queue_time': 0.249513349, 'total_time': 0.047149542}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--6dfea41e-130b-4734-b493-afde53a1326f-0', usage_metadata={'input_tokens': 92, 'output_tokens': 23, 'total_tokens': 115})

In [10]:
## Change the config it cant remember 
config1 = {"configurable": {"session_id": "chat2"}}
response = with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config1
)
response.content

'As an AI, I have no memory of past conversations and do not know your name. Can you tell me? 😊\n'

# Prompt Templates 

### prompt templates help to turn raw user info into a format that llm can work with , 

#### if raw is  just a message then it make more complicated first and then add in a system message with some custom instriction 

In [14]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all question to the  nest of your ability."),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [15]:
chain = prompt | model

In [16]:
chain.invoke({"messages":[HumanMessage(content="Hello my name is omkar i am a Ai engineering student")]})

AIMessage(content="Hello Omkar, it's nice to meet you! \n\nI understand you're an AI Engineering student. That's fascinating! What are you currently working on or interested in learning more about within AI engineering? \n\nI'm here to help in any way I can. Whether you have questions about specific concepts, need help brainstorming ideas, or just want to discuss the latest advancements in AI, feel free to ask.  \n\nLet's explore the world of AI together! 😊\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 104, 'prompt_tokens': 38, 'total_tokens': 142, 'completion_time': 0.189090909, 'prompt_time': 0.002403995, 'queue_time': 0.25409322500000003, 'total_time': 0.191494904}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--b1e71afa-6b47-4cd6-80bf-91fe43366ac3-0', usage_metadata={'input_tokens': 38, 'output_tokens': 104, 'total_tokens': 142})

In [17]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history
)

In [18]:
config = {"configurable": {"session_id": "chat3"}}
response = with_message_history.invoke(
    [HumanMessage(content="Hello my name is omkar i am a Ai engineering student")],
    config=config
)
response.content

"Hello Omkar! It's great to meet you. As an AI engineering student, you're diving into a fascinating field. \n\nWhat can I help you with today? \n\nAre you working on a particular project? Do you have any questions about AI concepts? Or perhaps you'd like to discuss the latest advancements in the field? \n\nI'm here to assist in any way I can. 😊  \n\n"

In [21]:
## Add more complexity 
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all question to the  nest of your ability in this {language}."),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model
response = chain.invoke({
    "messages":[HumanMessage(content="Hello my name is omkar i am a Ai engineering student")],
    "language":"Hindi"
})
response.content

'नमस्ते ओमकार! \n\nमैं आपकी मदद करने के लिए तैयार हूँ। एआइ इंजीनियरिंग में पढ़ाई करना कितना रोमांचक है! \n\nआपके किसी भी प्रश्न के लिए मैं यहाँ हूँ। \n\n'

In [23]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)


In [24]:
config = {"configurable": {"session_id": "chat4"}}
response = with_message_history.invoke(
   {
       "messages": [HumanMessage(content="Hello my name is omkar i am a Ai engineering student")],
       "language": "Hindi",
   }
   , config=config
)
response.content

'नमस्ते ओमकार! 😊\n\nयह जानकर बहुत खुशी हुई। तुम एआई इंजीनियरिंग में पढ़ाई कर रहे हो!  कौन सा विषय तुम्हें सबसे ज्यादा पसंद है? \n\nमैं तुम्हारी मदद करने के लिए तैयार हूँ।  \n\nक्या तुम कोई सवाल पूछना चाहते हो? \n\n'

## Manage the conversation history

In [ ]:
# if unmanaged the list of messages will grow unbounded and potentially overflow the context windows of the LLM.so we add the step the size of the context window


# trim_message -> it allow us to specify how many tokens we want to keep in the message history, and it will automatically trim the messages to fit within that limit.

In [30]:
from langchain_core.messages import SystemMessage,trim_messages

trimmer = trim_messages(
    max_tokens=1000,  # specify the maximum number of tokens to keep
    strategy="last",  # specify the strategy to use
    token_counter=model,
    include_system =True,  # include system messages in the trimming
    allow_partial=False,  
    start_on = "human"
)

messages = [
    SystemMessage(content="You are a helpful assistant. Answer all question to the  nest of your ability in this Hindi."),
    HumanMessage(content="Hello my name is omkar i am a Ai engineering student"),
    AIMessage(content="Hello Omkar, nice to meet you!"),
    HumanMessage(content="What is my name?"),
    AIMessage(content="Your name is Omkar."),
    HumanMessage(content="How are you?"),
    AIMessage(content="I am doing well, thank you for asking!"),
    HumanMessage(content="What is the weather today?"),
    AIMessage(content="The weather today is sunny."),
]

trimmed_messages = trimmer.invoke(messages)
trimmed_messages

[SystemMessage(content='You are a helpful assistant. Answer all question to the  nest of your ability in this Hindi.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello my name is omkar i am a Ai engineering student', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hello Omkar, nice to meet you!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Your name is Omkar.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='How are you?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='I am doing well, thank you for asking!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is the weather today?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The weather today is sunny.', additional_kwargs={}, response_metadata={})]

In [31]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(
        messages=itemgetter("messages") | trimmer # Extract the messages from the input
    )
    | prompt
    | model

)

response  = chain.invoke({
    "messages": messages + [HumanMessage(content="What is my name?")],
    "language": "Hindi",
})
response.content

'आपका नाम ओमकार है।  \n'

In [33]:
# lets wrarp this in the message History 
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)
config = {"configurable": {"session_id": "chat6"}}
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="What is my name?")],
        "language": "Hindi",
    },
    config=config
)
response.content

'आपका नाम Omkar है।  \n'